In [1]:
import cv2
import dlib
import numpy as np
#import vlc
import pygame
from scipy.spatial import distance 
import time

pygame 2.5.2 (SDL 2.28.3, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def eye_aspect_ratio(eye):
    a=distance.euclidean(eye[1],eye[5])
    b=distance.euclidean(eye[2],eye[4])
    c=distance.euclidean(eye[0],eye[3])
    res=(a+b)/(2.0*c+1e-7)
    return res

In [3]:
TIME_LIMIT=3*60*60
start_time=time.time()

In [4]:
face_detector=dlib.get_frontal_face_detector()
facial_point_predictor=dlib.shape_predictor(r"C:\Users\Nishith\Downloads\shape_predictor_68_face_landmarks.dat")

pygame.mixer.init()
pygame.mixer.music.load(r"C:\Users\Nishith\Downloads\FM9B3TC-alarm (1).mp3")
#pygame.mixer.music.load(r"C:\Users\Nishith\Downloads\alarm.mp3")
#sound=vlc.MediaPlayer(r"C:\Users\Nishith\Downloads\alarm.mp3")
sound=pygame.mixer.music

In [5]:
EAR_threshold=0.23

In [14]:
cap=cv2.VideoCapture(0)
frame_count=0
while True:
    ret,frame=cap.read()
    frame_count+=1
    if frame_count%5==0:
        img=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces=face_detector(img)
        for face in faces:
            facial_landmarks=facial_point_predictor(img,face)
            left_eye=np.array([(facial_landmarks.part(i).x,facial_landmarks.part(i).y )for i in range(36,42)],dtype="double")
            right_eye=np.array([(facial_landmarks.part(i).x,facial_landmarks.part(i).y )for i in range(42,48)],dtype="double")    
            left_eye_ar=eye_aspect_ratio(left_eye)
            right_eye_ar=eye_aspect_ratio(right_eye)
        
            if np.all(left_eye) and np.all(right_eye):
                res=(left_eye_ar+right_eye_ar)/2.0
            elif np.all(left_eye):
                res=left_eye_ar
            elif np.all(right_eye):
                res=right_eye_ar
            else:
                continue
            if res<0.20 :
                cv2.putText(frame,"DROWSYNESS DETECTED",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
                #if not sound.is_playing():
                if not sound.get_busy():
                    sound.play()
            else:
                #if sound.is_playing():
                if sound.get_busy():
                    sound.stop()
            time_difference=time.time()-start_time
            if time_difference>TIME_LIMIT:
                cv2.putText(frame,"more than 3 hours,TAKE BREAK",(10,60),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
                if not sound.get_busy():
                    sound.play()
                else:
                    if sound.get_busy():
                         sound.stop()
                    
                   
                    
        cv2.imshow('drowsiness',frame)
        if cv2.waitKey(1)==ord('q'):
            break
    
#if sound.is_playing():
if sound.get_busy():
    sound.stop()
cap.release()
cv2.destroyAllWindows()